In [37]:
import spotipy
import spotipy.util as util
import numpy as np
import pandas as pd
import os
import csv
import warnings
from sklearn.utils import shuffle

In [2]:
absolute_path = "f:\\AI Projects\\Music Classification"
relative_path = "Data\\Spotify"
full_path = os.path.join(absolute_path, relative_path)

In [3]:
CLIENT_ID = "372d9bfe8dce48a1aac1806d4329629d"
CLIENT_SECRET = "1a6e5a340428469dba631f9ffbd4528f"

In [4]:
auth_manager = spotipy.oauth2.SpotifyClientCredentials(client_id=CLIENT_ID,client_secret=CLIENT_SECRET)
sp = spotipy.Spotify(auth_manager=auth_manager)

Get tracks from playlists

In [32]:
playlists = [
"37i9dQZF1DWXRqgorJj26U", #Rock classics
"2MepDCzimtCz2ytAlLUdUi", #Mozart best of
"1h0CEZCm6IbFTbxThn6Xcs", #Best classical music
"3ZgmfR6lsnCwdffZUan8EA", #Top 100 pop tracks
"0K8DNHMwJaZmERK4FeaAOJ", #Best pop hits of all time
"4hOKQuZbraPDIfaGbM3lKI", #Top 100 tracks currently on spotify
"37i9dQZF1DWTWdbR13PQYH", #Top hits of 2004
"37i9dQZF1DX3j9EYdzv2N9", #Top hits of 2007
"37i9dQZF1DX4PrR66miO50", #Top hits of 1999
"37i9dQZF1DWWmGB2u14f8m", #Top hits of 1998
"37i9dQZF1DX6TtJfRD994c", #Top hits of 1991
"37i9dQZF1DXas7qFgKz9OV", #Top hits of 1982
"37i9dQZF1DWW2c0C8Vb2IR", #All new jazz
"37i9dQZF1DX9qNs32fujYe", #Heavy metal
"37i9dQZF1DWWOaP4H0w5b0", #Metal essentials
"37i9dQZF1DXd9rSDyQguIk", #Blues classics
"5E4CbUOCiUXw2Fh8Foq51V", #The best of classical music
"5E0T3gdmkcyzcqtmkCZRgf", #The best classicla music of all time
"3Di88mvYplBtkDBIzGLiiM", #EDM
"7qhhMMuWRxnq8pbvUlNcKy", #EDM & House top 100
"37i9dQZF1DX8qqIDAkKiQg", #This is mozart
"7pWNG89bI9uwHQaUI3pINb", #Beethoven best of
"37i9dQZF1DX12YIlzNkccL", #This is beethoven
"37i9dQZF1DWWEJlAGA9gs0", #Classical essentials
"37i9dQZF1DXbITWG1ZJKYt", #Jazz classics
"37i9dQZF1DX1spT6G94GFC", #80s rock anthems
"5oM9tyhspzK18TrPrlIZro", #80s rock hits music playlist
"37i9dQZF1DX55yuR78Invt", #This is harry styles
"35i05dxUnfnU0ulnimZh3V", #Ultimate Jazz playlist
"05Hd48jdQIz3s8WRrvGnzf", #Jazz playlist
"37i9dQZF1DWXb9I5xoXLjp", #The most romantic songs in the world
"4ckPlRonCUjkAhVnOXV6Ne", #The greatest love songs playlist
"37i9dQZF1DWSlwBojgQEcN", #Acoustic love
"2rN3mSrzUcgjlj1TcEDTX7", #Chill beats
"5QJ7ytA5oLJYnq9ihf907q", #Classical music: the 50 greatest tracks
"37i9dQZF1DX1kCIzMYtzum", #EDM #2
"37i9dQZF1DX3Kdv0IChEm9", #EDM hits
]

Define all artists

In [2]:
from itertools import chain
genre_count = {}
# broad_genres = [
#     'blues',
#     'classical',
#     'country',
#     'dance',
#     'electronic',
#     'folk',
#     'hip hop',
#     'rap',
#     'jazz',
#     'rock',
#     'r&b',
#     'soul',
#     'reggae',
#     'pop',
#     'soundtrack'
# ]

broad_genres = {
    'blues' : {'blues'},
    'classical' : {'classical','baroque','romantic','ragtime',},
    'country' : {'country'},
    'dance' : {'dance','edm'},
    'electronic' : {'electronic','edm'},
    'folk' : {'folk'},
    'hip hop' : {'hip hop',},
    'rap' : {'rap',},
    'jazz' : {'jazz'},
    'rock' : {'rock','metal','nwbm',''},
    'r&b' : {'r&b'},
    'soul' : {'soul'},
    'reggae' : {'reggae'},
    'pop' : {'pop','boy band'},
    'soundtrack' : {'soundtrack'}
}



    # 'rock',
    # 'metal',
    # 'pop',
    # 'rap',
    # 'contemporary',
    # 'hip hop',
    # 'r&b',
    # 'classical',
    # 'jazz',
    # 'reggae',
    # 'soul',
    # "edm",
    # "house",
    # "indie",
    # "wave",
    # "bass",
    # "disco",
    # "dance",
    # "blues",
    # "beats",
    # "electronic",
    # "baroque",
    # "romantic",
    # "ragtime",
    # "folk"

def get_broad_genres(genres):
    mapped_genres = []
    
    for g in genres:
        mapped_genres.append([g1 for g1 in broad_genres if g1 in g])

    mapped_genres = list(chain.from_iterable(mapped_genres))
    mapped_genres = mapped_genres = list(filter(None, mapped_genres))

    if mapped_genres: 
        mapped_genres = list(np.unique(mapped_genres))
        for g in mapped_genres:
            if g in genre_count:
                genre_count[g] = genre_count.get(g)  + 1
            else:
                genre_count[g] = 1
        return mapped_genres
    else:
        print("No broad genres found")
        return None


In [72]:
import warnings
warnings.filterwarnings('ignore')

def get_artists_from_playlist(playlist_url):
    columns = ["artist","uri","genres","broad_genres"]
    df = pd.DataFrame(columns=columns)
    total_artists = 0

    for playlist in playlist_url:
        playlist_data = sp.playlist_tracks(f"https://open.spotify.com/playlist/{playlist}")
        tracks = playlist_data['items'] 
        while playlist_data['next']:
            playlist_data = sp.next(playlist_data)
            tracks.extend(playlist_data['items'])

        for track in tracks:
            name = track["track"]["artists"][0]["name"]
            uri = track["track"]["artists"][0]["uri"]
            if not (df['artist'].str.contains(name, regex=False).any() & df['uri'].str.contains(uri, regex=False).any()):
                genres = sp.artist(uri)["genres"]
                if len(genres) != 0:
                    broad_genres = get_broad_genres(genres)
                    new_row = {"artist" : name, "uri" : uri, "genres" : genres, "broad_genres" : broad_genres}
                    df = df.append(new_row, ignore_index=True)
                    total_artists += 1
                else:
                    print(f"Skipping: {name} due no genre information available")
            else:
                print(f"Skipping: {name} due to a duplicate")

    print(f"Total number of artists: {total_artists}")
    return df

metadata_artists = get_artists_from_playlist(playlists)
metadata_artists.head()


Skipping: AC/DC due to a duplicate
Skipping: Fleetwood Mac due to a duplicate
Skipping: Queen due to a duplicate
Skipping: Van Halen due to a duplicate
Skipping: Bon Jovi due to a duplicate
Skipping: Pink Floyd due to a duplicate
Skipping: Van Halen due to a duplicate
Skipping: The Rolling Stones due to a duplicate
Skipping: Queen due to a duplicate
Skipping: Guns N' Roses due to a duplicate
Skipping: Led Zeppelin due to a duplicate
Skipping: ZZ Top due to a duplicate
Skipping: Bruce Springsteen due to a duplicate
Skipping: The Police due to a duplicate
Skipping: Creedence Clearwater Revival due to a duplicate
Skipping: AC/DC due to a duplicate
Skipping: David Bowie due to a duplicate
Skipping: Eagles due to a duplicate
Skipping: Red Hot Chili Peppers due to a duplicate
Skipping: Lynyrd Skynyrd due to a duplicate
Skipping: Nirvana due to a duplicate
Skipping: Fleetwood Mac due to a duplicate
Skipping: U2 due to a duplicate
Skipping: Foo Fighters due to a duplicate
Skipping: The Who due

,artist,uri,genres,broad_genres
0,Aerosmith,spotify:artist:7Ey4PD4MYsKc5I2dolUwbH,"[album rock, classic rock, hard rock, rock]",[rock]
1,Heart,spotify:artist:34jw2BbxjoYalTp8cJFCPv,"[album rock, classic rock, hard rock, heartlan...","[pop, rock, wave]"
2,Foo Fighters,spotify:artist:7jy3rLJdDQY21OgRLCZ9sD,"[alternative metal, alternative rock, modern r...","[metal, rock, wave]"
3,Bon Jovi,spotify:artist:58lV9VcRSjABbAbfWS6skp,"[glam metal, rock]","[metal, rock]"
4,ZZ Top,spotify:artist:2AM4ilv6UzW0uMRuqKtDgN,"[album rock, blues rock, classic rock, country...","[blues, metal, rock]"


In [73]:
genre_count

{'rock': 439,
 'pop': 661,
 'wave': 127,
 'metal': 161,
 'blues': 100,
 'folk': 106,
 'indie': 96,
 'dance': 313,
 'romantic': 101,
 'hip hop': 105,
 'rap': 159,
 'classical': 131,
 'baroque': 11,
 'contemporary': 164,
 'jazz': 214,
 'edm': 146,
 'house': 161,
 'r&b': 68,
 'soul': 106,
 'electronic': 19,
 'reggae': 20,
 'disco': 42,
 'bass': 12,
 'beats': 36,
 'ragtime': 2}

In [74]:
metadata_artists.to_csv(os.path.join(full_path,"metadata_artists_complete_2.csv"),index=False)

Get top tracks from artists

In [6]:
metadata_artists = pd.read_csv(os.path.join(full_path,"metadata_artists_complete.csv")) 

# metadata_artists = shuffle(metadata_artists).reset_index(drop=True)
# metadata_artists = metadata_artists.sample(frac=0.3)

artists_col = metadata_artists.loc[:,"artist"]
uri_col = metadata_artists.loc[:,"uri"]
broad_genre_col = metadata_artists.loc[:,"broad_genres"]

In [11]:
metadata_artists.shape

(2201, 4)

In [12]:
warnings.filterwarnings('ignore')

columns = ["artist","album_name","track_name","track_id","released_date","explicit","genres","danceability","energy","key","loudness","mode","speechiness","acousticness","instrumentalness","liveness","valence","tempo","duration_ms","time_signature"]
metadata_toptracks = pd.DataFrame(columns=columns)
total_tracks = 0

for i in range(0,len(metadata_artists)):
    artist = artists_col[i]
    artist_uri = uri_col[i]
    genres = broad_genre_col[i]
    results = sp.artist_top_tracks(artist_uri) 
    print(f"Processing top tracks by {artist}")
    for track in results["tracks"][:3]:
        album_name = track["album"]["name"]
        track_name = track["name"]
        if not (metadata_toptracks['artist'].str.contains(artist, regex=False).any() & 
        metadata_toptracks['album_name'].str.contains(album_name, regex=False).any() & 
        metadata_toptracks['track_name'].str.contains(track_name, regex=False).any()):
            track_id = track["id"]
            released_date = track["album"]["release_date"]
            explicit = track["explicit"]
            new_row = {"artist" : artist, "album_name" : album_name, "track_name" : track_name, "track_id" : track_id,"released_date" : released_date, "explicit" : explicit, "genres" : genres}
            audio_features = sp.audio_features(track_id)[0]
            for feature in columns[7:]:
                if audio_features is None:
                    print(f"Skipping: {track_name} due to no audio features")
                    break
                new_row[feature] = audio_features[feature]
            if audio_features is not None:
                metadata_toptracks = metadata_toptracks.append(new_row, ignore_index=True)
                total_tracks += 1
        else:
            print(f"Skipping: {track_name} due to a duplicate")
print(f"Total number of tracks processed: {total_tracks}")

Processing top tracks by Aerosmith
Processing top tracks by Heart
Processing top tracks by Foo Fighters
Processing top tracks by Bon Jovi
Processing top tracks by ZZ Top
Processing top tracks by David Bowie
Processing top tracks by The White Stripes
Processing top tracks by Fleetwood Mac
Processing top tracks by AC/DC
Processing top tracks by Def Leppard
Processing top tracks by Eagles
Processing top tracks by Alice In Chains
Processing top tracks by Led Zeppelin
Processing top tracks by Radiohead
Processing top tracks by Van Halen
Processing top tracks by Judas Priest
Processing top tracks by Queen
Processing top tracks by Bruce Springsteen
Processing top tracks by Metallica
Processing top tracks by Dire Straits
Processing top tracks by Pearl Jam
Skipping: Black due to a duplicate
Processing top tracks by Peter Gabriel
Processing top tracks by Mötley Crüe
Processing top tracks by U2
Processing top tracks by Creedence Clearwater Revival
Processing top tracks by The Police
Processing to

In [13]:
metadata_toptracks.shape

(6231, 20)

In [14]:
metadata_toptracks.head()

,artist,album_name,track_name,track_id,released_date,explicit,genres,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
0,Aerosmith,Aerosmith,Dream On,1xsYj84j7hUDDnTTerGWlH,1973-01-05,False,['rock'],0.392,0.476,1,-7.583,1,0.0249,0.3970,0.000163,0.3090,0.234,78.031,267597,4
1,Aerosmith,I Don't Want To Miss A Thing EP,"I Don't Want To Miss A Thing - From ""Armageddo...",5qlJg0Wbj3sslxfwiaRNpq,1998-08-18,False,['rock'],0.390,0.649,2,-5.163,1,0.0345,0.1720,0.000000,0.2160,0.144,121.161,299723,4
2,Aerosmith,Get A Grip,Crazy,74irxdVWstNlEQjsvArITq,1993-01-01,False,['rock'],0.370,0.866,6,-3.879,0,0.0384,0.0868,0.000000,0.1370,0.427,162.086,316693,3
3,Heart,Little Queen,Barracuda,4KfSdst7rW39C0sfhArdrz,1977-05-14,False,['pop' 'rock' 'wave'],0.548,0.691,4,-13.553,0,0.0369,0.0150,0.106000,0.1440,0.667,137.145,261933,4
4,Heart,Bad Animals,Alone,54b8qPFqYqIndfdxiLApea,1987-06-06,False,['pop' 'rock' 'wave'],0.418,0.452,1,-13.099,1,0.0356,0.6380,0.000260,0.0959,0.168,175.088,218733,4


In [19]:
metadata_toptracks.to_csv(os.path.join(full_path,"metadata_top3tracks_complete.csv"),index=False)

In [23]:
metadata_toptracks.to_excel(os.path.join(full_path,"metadata_top3tracks_complete.xlsx"),index=False)